<a href="https://www.kaggle.com/code/joymachadolab/brandmr-demo?scriptVersionId=248515087" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip uninstall torch -y

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121


In [2]:
! pip install --upgrade unsloth --no-cache-dir 
! pip install --upgrade unsloth-zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 150.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 270.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 311.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 281.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 275.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 288.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 366.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 277.1 MB/s eta

In [3]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Modules for fine-tuning
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
from kaggle_secrets import UserSecretsClient
import pandas as pd


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf = user_secrets.get_secret("huggingface")
wnb_token = user_secrets.get_secret("wandb")
login(hf)
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on BrandMR signup form',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: j-machado (j-machado-lab-digital). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
df = pd.read_excel("../input/casetype-dataset/casetype_dataset.xlsx")
column_values = set(df["Zaaktype benaming"])
case_type_list = ", ".join(map(str, column_values))

In [8]:
# Define a system prompt under prompt_style
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an intake employee for a Dutch legal firm expert with advanced knowledge in assigning incoming requests from users to the right kind of case types set by the firm.
The case types set by the legal firm are listed as: {}
Answer the following question sent in by a user in the Dutch language and classify the user question to the right kind of case type set by the firm along with your chain of thought. 
If there is more than 1 case type that's relevant to a situation, pick the one that is 
most relevant by reasoning why and stick to the case types set by the firm.

### Question:
{}

### Response:
<think>{}"""# Define a system prompt under prompt_style and use it to train

In [9]:
# Define a system prompt under prompt_style and use it to train
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an intake employee for a Dutch legal firm expert with advanced knowledge in assigning incoming requests from users to the right kind of case types set by the firm.
The case types set by the legal firm are listed as: {}
Answer the following question sent in by a user in the Dutch language and classify the user question to the right kind of case type set by the firm along with your chain of thought. 
If there is more than 1 case type that's relevant to a situation, pick the one that is 
most relevant by reasoning why and stick to the case types set by the firm.

### Question:
{}

### Response:
{}
"""

In [11]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(example, case_type_list):
  inputs = example["Beschrijving het probleem"]
  outputs = example["Zaaktype benaming"]
  texts = []
  for input, output in zip(inputs, outputs):
      text = train_prompt_style.format(case_type_list, input, output) + EOS_TOKEN
      texts.append(text)
  return {
      "text": texts,
  }
 
dataset = load_dataset("jmachado-lab/brandmr_casetype_dataset", split = "train[0:300]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True, fn_kwargs={'case_type_list': case_type_list})
dataset["text"][0]

Map:   0%|          | 0/297 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are an intake employee for a Dutch legal firm expert with advanced knowledge in assigning incoming requests from users to the right kind of case types set by the firm.\nThe case types set by the legal firm are listed as: nan, CA Dier, VK Aanrijding eenvoudige behandeling, CA Faillissement, CA Verzekeringen, AS ARBEIDSRECHT Concurrentiebeding, WO ZAKELIJK RECHT Eigendom/opstal, VK Aanrijding waarborgfonds, CA Pleziervaartuig/vliegtuig, AS ARBEIDSRECHT Functiewijziging, WO BESTUURSRECHT Ruimtelijke ordening/omgevingsplan, VK Nederlands bureau (aanrijding in NL met bl.), LS ONRECHTMATIGE DAAD verkeer Zwaar, AS ARBEIDSRECHT Pensioen, LS MISHANDELING Middel, AS ARBEIDSRE

In [12]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quan
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
trainer = SFTTrainer(
    model=model_lora, # Use the LoRA model
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/297 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 297 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.956700
20,0.403100
30,0.062400
40,0.049700
50,0.042600
60,0.039900


In [15]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,▅█▁▁▁▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▁▁▁▁
total_flos,2.789123069047603e+16
train/epoch,1.5906
train/global_step,60
train/grad_norm,0.11903
train/learning_rate,0
train/loss,0.0399


In [16]:
FastLanguageModel.for_inference(model_lora)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [26]:
question = "Ik was een werknemer in een winkel en ik werd beschuldigd van diefstal en ontslagen van mijn baan. Ik heb niets gestolen, daar ben ik zeker van. Kunt u mij helpen?"

inputs = tokenizer([train_prompt_style.format(case_type_list, question, "")], return_tensors="pt").to("cuda")

outputs = model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0])

["<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are an intake employee for a Dutch legal firm expert with advanced knowledge in assigning incoming requests from users to the right kind of case types set by the firm.\nThe case types set by the legal firm are listed as: nan, CA Dier, VK Aanrijding eenvoudige behandeling, CA Faillissement, CA Verzekeringen, AS ARBEIDSRECHT Concurrentiebeding, WO ZAKELIJK RECHT Eigendom/opstal, VK Aanrijding waarborgfonds, CA Pleziervaartuig/vliegtuig, AS ARBEIDSRECHT Functiewijziging, WO BESTUURSRECHT Ruimtelijke ordening/omgevingsplan, VK Nederlands bureau (aanrijding in NL met bl.), LS ONRECHTMATIGE DAAD verkeer Zwaar, AS ARBEIDSRECHT Pensioen, LS MISHANDELIN

In [27]:
new_model_online = "jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification"
model_lora.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)

model_lora.push_to_hub_merged(new_model_online, tokenizer)

README.md:   0%|          | 0.00/637 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification


No files have been modified since last commit. Skipping to prevent empty commit.
Unsloth: You are pushing to hub in Kaggle environment.
To save memory, we shall move jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification to /tmp/DeepSeek-R1-BrandMR-casetype-classification
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


...

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/jmachado-lab/DeepSeek-R1-BrandMR-casetype-classification
